# How happy are Zürcher with Tempo 30?

Dataset from [Daten der Bevölkerungsbefragung der Stadt Zürich 1999 bis 2019](https://data.stadt-zuerich.ch/dataset/prd_stez_bevoelkerungsbefragungen_seit1999_od4730)

We want to compare this with the kilometers (or number of roads) with Tempo 30 in each Kreis :
- [Verkehrszonen](https://www.ogd.stadt-zuerich.ch/geodaten/Verkehrszonen?format=10009)

---

You need to unzip `bev473od4731.zip` before running this notebook.

In [104]:
import pandas as pd

In [105]:
df = pd.read_csv('bev473od4731.csv')
df.shape

(28370, 318)

In [106]:
df.head()

,StichtagDatJahr,intnr,methode,fbversion,sgewicht,pgewicht,kreis,s1,f1,f2,...,f58_8,f58_9,f58_10,f58_11,f58_12,f58_13,f59,f60a,f60b,f60c
0,2019,1101,online,Unbekannt,0.279589,17.750000,Kreis 01/02,Frau,Eher gerne,Note 4,...,Ja,Nein,Nein,Nein,Nein,Nein,Sehr zufrieden (6),Sehr zufrieden (6),Sehr zufrieden (6),Sehr zufrieden (6)
1,2019,1102,online,Unbekannt,1.020983,64.818182,Kreis 07,Frau,Eher gerne,Note 5,...,Nein,Nein,Nein,Nein,Nein,Nein,3,4,3,3
2,2019,1103,online,Unbekannt,0.551302,35.000000,Kreis 01/02,Frau,Sehr gerne,Note 6 (sehr gut),...,Ja,Nein,Nein,Nein,Nein,Nein,Sehr zufrieden (6),4,5,5
3,2019,1104,online,Unbekannt,0.989136,62.796296,Kreis 09,Frau,Sehr ungern,Note 6 (sehr gut),...,Nein,Nein,Nein,Nein,Nein,Nein,4,5,5,5
4,2019,1105,online,Unbekannt,0.986286,62.615385,Kreis 01/02,Frau,Sehr gerne,Note 5,...,Nein,Nein,Nein,Nein,Ja,Nein,3,3,3,4


- [Renaming columns](https://stackoverflow.com/a/46096346/5239250)

In [114]:
happy = df[['kreis', 'f1', 'f4l']].rename(columns={'f1': 'happy_zh', 'f4l': 'happy_30'})

happy.head()

,kreis,happy_zh,happy_30
0,Kreis 01/02,Eher gerne,Weiss nicht
1,Kreis 07,Eher gerne,Weiss nicht
2,Kreis 01/02,Sehr gerne,Note 4
3,Kreis 09,Sehr ungern,Note 6 (sehr zufrieden)
4,Kreis 01/02,Sehr gerne,Note 5


https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/

In [115]:
happy.happy_zh.unique()

array(['Eher gerne', 'Sehr gerne', 'Sehr ungern', 'Eher ungern',
       'Keine Angabe'], dtype=object)

In [116]:
happy = happy[happy.happy_zh != 'Keine Angabe']
categories = {'happy_zh': {'Sehr gerne': 4, 'Eher gerne': 3, 'Eher ungern': 2, "Sehr ungern": 1}}
happy.replace(categories, inplace=True)
happy.head()

,kreis,happy_zh,happy_30
0,Kreis 01/02,3,Weiss nicht
1,Kreis 07,3,Weiss nicht
2,Kreis 01/02,4,Note 4
3,Kreis 09,1,Note 6 (sehr zufrieden)
4,Kreis 01/02,4,Note 5


- [How do I remove unwanted parts from strings in a column?](https://stackoverflow.com/a/54302517/5239250)

Removing all but the digits leaves us with a number as string, which we have to convert to numeric.

In [117]:
happy.happy_30.unique()

array(['Weiss nicht', 'Note 4', 'Note 6 (sehr zufrieden)', 'Note 5',
       'Note 2', 'Note 3', 'Note 1 (sehr unzufrieden)', 'Keine Angabe'],
      dtype=object)

In [119]:
happy = happy[happy.happy_30 != "Weiss nicht"]
happy = happy[happy.happy_30 != "Keine Angaben"]
happy.happy_30 = happy.happy_30.str.extract('(\d+)', expand=False)
happy.happy_30 = pd.to_numeric(happy.happy_30)
happy


,kreis,happy_zh,happy_30
2,Kreis 01/02,4,4.0
3,Kreis 09,1,6.0
4,Kreis 01/02,4,5.0
6,Kreis 10,4,4.0
7,Kreis 11,4,2.0
...,...,...,...
28365,Kreis 12,3,5.0
28366,Kreis 12,4,6.0
28367,Kreis 12,4,6.0
28368,Kreis 12,4,4.0


In [120]:
happy.groupby('kreis').mean()

,happy_zh,happy_30
kreis,,
Kreis 01/02,3.757342,4.309773
Kreis 03,3.751210,4.470559
Kreis 04,3.702027,4.521555
Kreis 05,3.753425,4.518968
Kreis 06,3.783482,4.506131
Kreis 07,3.778970,4.325750
Kreis 08,3.803717,4.248266
Kreis 09,3.708949,4.296989
Kreis 10,3.777013,4.337150


happy_zh: 1-4; happy_30: 1-6

In [95]:
df = pd.read_json('taz.vz_tbl_tempo_l.json')

In [96]:
from pandas.io.json import json_normalize
strassen = json_normalize(df['features'])
strassen.head()

,type,geometry.type,geometry.coordinates,properties.objid,properties.messwert_von,properties.messwert_bis,properties.vmax,properties.umsetzung,properties.laenge,properties.vmax_soll_grund,properties.gemeldetvon,properties.lokalisationsname,properties.lokalisationnummer,properties.t30_typ
0,Feature,LineString,"[[8.5277852333, 47.4035998024], [8.5277102315,...",3497804,0.000000,1300.821870,0,None,None,None,None,Reitweg,1602,1
1,Feature,LineString,"[[8.4758567946, 47.4132868951], [8.4757936448,...",3497805,378.064924,925.832900,0,None,None,None,None,Rütihofstrasse,1692,1
2,Feature,LineString,"[[8.5336153527, 47.3892984469], [8.5344402187,...",3497806,0.000000,81.042079,0,None,None,None,None,Moosbergerweg,1388,1
3,Feature,LineString,"[[8.5106903026, 47.4270367954], [8.5107730511,...",3497807,185.000471,332.707923,80,None,None,None,None,Bärenbohlstrasse,159,1
4,Feature,LineString,"[[8.4923007552, 47.4262873771], [8.4918118298,...",3497808,19974.437428,20157.823834,100,None,None,None,None,A1,4411,1


In [97]:
strassen.iloc[1]['geometry.coordinates']

[[8.4758567946, 47.4132868951],
 [8.4757936448, 47.4132783586],
 [8.4752325832, 47.4132103253],
 [8.4750597894, 47.4131893785],
 [8.4747896982, 47.4131558649],
 [8.4746255954, 47.413135505],
 [8.4735822808, 47.4130189299],
 [8.4725559009, 47.4128788705],
 [8.4720242967, 47.4127987386],
 [8.470555984, 47.4125888258],
 [8.4700319011, 47.4125108664],
 [8.4687607893, 47.4122671312]]

In [98]:
# https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas
import numpy as np
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

haversine_np(first[0][0], first[0][1], first[1][0], first[1][1])*1000



4.842620155129374

In [99]:
print(strassen.iloc[1]['properties.lokalisationsname'])
df_coord = pd.DataFrame(strassen.iloc[1]['geometry.coordinates'])
# df_coord = df_coord.transpose()
df_coord.columns = ['long', 'lat']
df_coord

Rütihofstrasse


,long,lat
0,8.475857,47.413287
1,8.475794,47.413278
2,8.475233,47.413210
3,8.475060,47.413189
4,8.474790,47.413156
5,8.474626,47.413136
6,8.473582,47.413019
7,8.472556,47.412879
8,8.472024,47.412799
9,8.470556,47.412589


In [100]:
df_coord['dist'] = \
    haversine_np(df_coord.long.shift(), df_coord.lat.shift(),
                 df_coord.loc[1:, 'long'], df_coord.loc[1:, 'lat'])
df_coord

,long,lat,dist
0,8.475857,47.413287,NaN
1,8.475794,47.413278,0.004843
2,8.475233,47.413210,0.042863
3,8.475060,47.413189,0.013201
4,8.474790,47.413156,0.020649
5,8.474626,47.413136,0.012546
6,8.473582,47.413019,0.079519
7,8.472556,47.412879,0.078737
8,8.472024,47.412799,0.040956
9,8.470556,47.412589,0.112854


In [101]:
# https://www.google.com/maps/dir/'47.413287,8.475857'/'47.412267,8.468761'
df_coord.dist.sum()

0.5458707437053021